# Homework 4

# Ⅰ

In [1]:
import numpy as np

t = np.random.normal(0, 50, 50)
X = np.array([np.ones_like(t), t, t**2]).T

def func(a, b, c, mu, sigma):
    return lambda x: a + b * x + c * x**2 + np.random.normal(mu, sigma, x.size)

f = func(10, 1, 2, 0, 0.01)
Y = f(t)

In [7]:
# \beta_1的t值
from sklearn.linear_model import LinearRegression 
model = LinearRegression(fit_intercept=False)
model.fit(X, Y)
beta_1 = model.coef_[1]

sigma = (Y - model.predict(X)).std(ddof=3)
model_2 = LinearRegression(fit_intercept=False)
model_2.fit(X[:, [0,2]], X[:, 1])
R2 = model_2.score(X[:, [0,2]], X[:, 1])

In [31]:
from scipy import stats
# H0: \beta_1 = 0  
t = beta_1 / (sigma / X[:, 1].std() / np.sqrt(X.shape[0] * (1 - R2)))
# 根据显著水平计算临界值
alpha = 0.05
t_critical = stats.t.ppf(1 - alpha / 2, df=X.shape[0] - 3)
print('t值为：', t, '临界值为：', t_critical)

# 根据计算值给出显著水平
p = stats.t.sf(np.abs(t), df=X.shape[0] - 3) * 2
print('p值为：', p, '显著水平为：', alpha)

print("故拒绝原假设，认为beta_1不等于0")

t值为： 41210.38065847728 临界值为： 2.0117405104757546
p值为： 2.8351052596617946e-179 显著水平为： 0.05
故拒绝原假设，认为beta_1不等于0


In [32]:
# H0: \beta_1 = 1
t = (beta_1 - 1) / (sigma / X[:, 1].std() / np.sqrt(X.shape[0] * (1 - R2)))
# 根据显著水平计算临界值
alpha = 0.05
t_critical = stats.t.ppf(1 - alpha / 2, df=X.shape[0] - 3)
print('t值为：', t, '临界值为：', t_critical)

# 根据计算值给出显著水平
p = stats.t.sf(np.abs(t), df=X.shape[0] - 3) * 2
print('p值为：', p, '显著水平为：', alpha)

print("故接受原假设，认为beta_1等于1")

t值为： -0.7782161147739707 临界值为： 2.0117405104757546
p值为： 0.4403418875923616 显著水平为： 0.05
故接受原假设，认为beta_1等于1


In [34]:
# 拟合优度
SSR = ((model.predict(X) - Y)**2).sum()
SST = ((Y - Y.mean())**2).sum()
uSST = (Y**2).sum()
print('Rc为：', 1 - SSR / SST)
print('Ruc为：', 1 - SSR / uSST)
print('Adjusted Rc为：', 1 - (SSR/(X.shape[0] - 3)) / (SST/(X.shape[0] - 1)))

print('残差平方和为：', SSR)
print('sigma^2的无偏估计为：', SSR / (X.shape[0] - 3))

Rc为： 0.9999999999990038
Ruc为： 0.9999999999992728
Adjusted Rc为： 0.9999999999989614
残差平方和为： 0.003494825304884599
sigma^2的无偏估计为： 7.435798521031061e-05


In [36]:
import statsmodels.api as sm
sm_model = sm.OLS(Y, X).fit()
sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.359e+13
Date:                Thu, 05 Oct 2023   Prob (F-statistic):          9.15e-283
Time:                        16:35:13   Log-Likelihood:                 168.27
No. Observations:                  50   AIC:                            -330.5
Df Residuals:                      47   BIC:                            -324.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.0022      0.001   7008.954      0.000       9.999      10.005
x1             1.0000   2.43e-05   4.12e+04      0.000       1.000       1.000
x2             2.0000   2.92e-07   6.86e+06      0.000       2.000       2.000
==============================================================================
Omnibus:                        0.597   Durbin-Watson:                   2.124
Prob(Omnibus):                  0.742   Jarque-Bera (JB):                0.207
Skew:                           0.145   Prob(JB):                        0.902
Kurtosis:                       3.124   Cond. No.                     5.73e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
# 使用的是Rc，当然也有adjusted Rc
print(sm_model.rsquared)
print(sm_model.rsquared_adj)

0.9999999999990038
0.9999999999989614


# Ⅱ

In [48]:
x = np.linspace(-50, 50, 101)
y = 1 + 0.5 * x + np.random.normal(0, 1, 101)

## H0: alpha_1 + alpha_2 = 1.5

基于Wald统计量，我们知道：
$$
(1^T \alpha - 1.5)^T (Var(1^T \alpha))^{-1} (1^T \alpha - 1.5) \sim \chi^2(1)
$$
其中：
$$
1 = \begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$
$$
\alpha = \begin{bmatrix} \alpha_2 \\ \alpha_2 \end{bmatrix}
$$

同时我们知道：
\begin{align*}
Var(1^T \alpha) &= 1^T Var(\alpha) 1 \\
&= \sigma^2 1^T(X^T X)^{-1} 1 \\
\end{align*} 

且我们可以估计$\sigma^2$：
$$
\hat{\sigma}^2 = \frac{1}{n-2} \sum_{i=1}^n (y_i - \hat{y}_i)^2
$$
更进一步，我们知道：
$$
\frac{(n-2)\hat{\sigma}^2 }{\sigma^2} \sim \chi^2(n-2)
$$

所以有：
$$
\frac{
    (1^T \alpha - 1.5)^T (1^T(X^T X)^{-1} 1)^{-1} (1^T \alpha - 1.5) / 1
}{
    (n-2)\hat{\sigma}^2 / (n-2)
} \sim F(1, n-2)



In [86]:
# 代入数据
X = np.array([np.ones_like(x), x]).T
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
alpha = model.coef_
sigma2_hat = ((y - model.predict(X))**2).sum() / (X.shape[0] - 2)

F_1 = (alpha.sum() - 1.5)**2 / (np.linalg.inv(X.T @ X).sum()) / sigma2_hat

若是用约束求解问题来看，不妨只考虑带约束的情形：
$$
\begin{align*}
\min_{\alpha} \quad & ||Y - X\alpha||^2 \\
s.t. \quad & R \alpha = q
\end{align*}
$$

使用Lagrange乘子法，我们有：
$$
\mathcal{L}(\alpha, \lambda) = ||Y - X\alpha||^2 + (R \alpha - q)^T \lambda
$$

求导得：
\begin{cases}
\frac{\partial \mathcal{L}}{\partial \alpha} &= -2 X^T Y + 2 X^T X \alpha + R ^T\lambda = 0 \\
\frac{\partial \mathcal{L}}{\partial \lambda} &= R \alpha - q = 0
\end{cases}

将1式两侧同时左乘$\frac{1}{2}R(X^T X)^{-1}$并，得：
$$
q = R\alpha = R(X^T X)^{-1} X^T Y - \frac{1}{2} R(X^T X)^{-1} R^T \lambda
$$

进而解得：
$$
\frac{\lambda}{2} = (R(X^T X)^{-1} R^T)^{-1} (R(X^T X)^{-1} X^T Y - q)
$$

代回1式，得：
$$
\alpha = (X^T X)^{-1} X^T Y - (X^T X)^{-1} R^T (R(X^T X)^{-1} R^T)^{-1} (R(X^T X)^{-1} X^T Y - q)
$$

如果我们用$\beta$表示无约束估计的解，那么上式可以写成：
$$
\alpha = \beta - (X^T X)^{-1} R^T (R(X^T X)^{-1} R^T)^{-1} (R\beta - q)
$$

在该估计下的SSR为：
$$
\begin{align*}
SSR(\alpha) &= (Y - X\alpha)^T (Y - X\alpha) \\
&= (Y - X\beta + X\beta - X\alpha)^T (Y - X\beta + X\beta - X\alpha) \\
&= (Y - X\beta)^T (Y - X\beta) + (X\beta - X\alpha)^T (X\beta - X\alpha) \\
&= SSR(\beta) + (X\beta - X\alpha)^T (X\beta - X\alpha) \\
&= SSR(\beta) + (\beta - \alpha)^T X^T X (\beta - \alpha) \\
\end{align*}
$$

我们发现有：
$$
\begin{align*}
SSR(\alpha) - SSR(\beta) &= (\beta - \alpha)^T X^T X (\beta - \alpha) \\
&= ((X^T X)^{-1} R^T (R(X^T X)^{-1} R^T)^{-1} (R\beta - q))^T X^T X ((X^T X)^{-1} R^T (R(X^T X)^{-1} R^T)^{-1} (R\beta - q)) \\
&= (R\beta - q)^T (R(X^T X)^{-1} R^T)^{-1} R (X^T X)^{-1} R^T (R(X^T X)^{-1} R^T)^{-1} (R\beta - q) \\
&= (R\beta - q)^T (R(X^T X)^{-1} R^T)^{-1} (R\beta - q) \\
\end{align*}
$$

故方法1中的Wald统计量可以写成：
$$
F= \frac{(SSR(\alpha) - SSR(\beta)) / J}{\hat{\sigma}^2} = \frac{(SSR(\alpha) - SSR(\beta)) / J}{SSR(\beta) / (n - p)} 
= \frac{R^2(\beta) - R^2(\alpha)}{1 - R^2(\beta)} \frac{n - p}{J}


In [84]:
# 代入数据
model_2 = LinearRegression(fit_intercept=False)
X_2 = x - 1
Y_2 = y - 1.5
model_2.fit(X_2.reshape(-1, 1), Y_2)

R2_1 = model.score(X, y)
R2_2 = model_2.score(X_2.reshape(-1, 1), Y_2)
F_2 = (R2_1 - R2_2) / (1 - R2_1) * (x.size - 2)

In [90]:
print('F_1为：', F_1)
print('F_2为：', F_2)
print('在显著性水平为5%时，临界值为：', stats.f.ppf(1 - 0.05, 1, x.size - 2), '故接受原假设')


F_1为： 0.05581708146678345
F_2为： 0.05581708146752701
在显著性水平为5%时，临界值为： 3.937116910962888 故接受原假设


## H0: alpha_1 + alpha_2 = 10

In [91]:
# 代入数据
X = np.array([np.ones_like(x), x]).T
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
alpha = model.coef_
sigma2_hat = ((y - model.predict(X))**2).sum() / (X.shape[0] - 2)

F_1 = (alpha.sum() - 10)**2 / (np.linalg.inv(X.T @ X).sum()) / sigma2_hat

In [92]:
# 代入数据
model_2 = LinearRegression(fit_intercept=False)
X_2 = x - 1
Y_2 = y - 10
model_2.fit(X_2.reshape(-1, 1), Y_2)

R2_1 = model.score(X, y)
R2_2 = model_2.score(X_2.reshape(-1, 1), Y_2)
F_2 = (R2_1 - R2_2) / (1 - R2_1) * (x.size - 2)

In [94]:
print('F_1为：', F_1)
print('F_2为：', F_2)
print('在显著性水平为10%时，临界值为：', stats.f.ppf(1 - 0.1, 1, x.size - 2), '故拒绝原假设')

F_1为： 6964.242508649678
F_2为： 6964.242508649698
在显著性水平为10%时，临界值为： 2.756898790979433 故拒绝原假设


# Ⅲ

In [97]:
import pandas as pd
data = pd.read_csv('D:\study\Math\时间序列分析\Data\iris_dataset.csv')

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split    
from sklearn.metrics import confusion_matrix, classification_report

model = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=0)
model.fit(X_train, y_train)

print('混淆矩阵为：', confusion_matrix(y_test, model.predict(X_test)))
print('分类报告为：\n', classification_report(y_test, model.predict(X_test)))


混淆矩阵为： [[11  0  0]
 [ 0 13  0]
 [ 0  1  5]]
分类报告为：
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        11
         1.0       0.93      1.00      0.96        13
         2.0       1.00      0.83      0.91         6

    accuracy                           0.97        30
   macro avg       0.98      0.94      0.96        30
weighted avg       0.97      0.97      0.97        30



# Ⅳ

In [114]:
data = pd.read_csv("D:\study\Math\时间序列分析\Data\MCdata_SH(1).csv")
GDP = data.iloc[:, 1]
CPI = data.iloc[:, 2]
Invest = data.iloc[:, 3]
R = data.iloc[:, 4]

T = np.arange(1, GDP.size + 1)
G = GDP / CPI
Y = Invest / CPI
P = CPI.pct_change()
P[0] = 0

In [118]:
X = np.array([T, G, R, P]).T
Y = Y.values.reshape(-1, 1)

In [141]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

model = Pipeline([
    ('scaler', StandardScaler()),
    ('regression', Lasso(alpha=3e-4, random_state=44))
])
model.fit(X, Y)
print("模型的R方为：", model.score(X, Y))

模型的R方为： 0.9587127838564551
